In [1]:
import os
import sys

import pandas as pd
import numpy as np
import datetime

from django.db import IntegrityError

from tv import models as tv_models


# Change DataFrame dtypes to match Database models
def model_for_db(csv):
    """
    @parameters:
    - csv: Electronic Guide filename in comma separated values (.csv)

    @returns:
    - Dataframe with items formatted for bmat tv/av programs
    """

    # Load csv into Dataframe
    df = pd.read_csv(csv)

    # Fill n/a with readable string "n/a"
    df = df.fillna('n/a')
    df['program_year'] = df['program_year'].astype(str)
    df['start_time'] = (df['start_date'].astype(str) + ' ' + df['start_time']).map(
        lambda x: datetime.datetime.strptime(x, '%Y%m%d %H:%M')
    )
    df['duration_in_seconds'] = pd.to_timedelta(df['duration_in_seconds'], unit='seconds')
    df['end_time'] = df['start_time'] + df['duration_in_seconds']
    df = df.drop(['start_date', 'duration_in_seconds'], axis=1)
    return df


# Get or create Program with channel obj and row data
def create_program(row, channel):
    if row.program_year != 'n/a':
        year = row.program_year[0:4]
    else:
        year = row.program_year
    program, created =tv_models.Program.objects.get_or_create(uid=row.program_id, channel=channel,
                                                            year=year,
                                                            local_title=row.program_local_title,
                                                            original_title=row.program_original_title
                                                            )
    if created:
        program.save()
        print("Program {} with id {} added to DB.".format(program.local_title,
                                                          program.uid,
                                                          ))
    return program


def create_channel(row):
    channel, created = tv_models.Channel.objects.get_or_create(uid=row.channel_id,
                                                     name=row.channel_name,
                                                     country_code=row.channel_country
                                                     )
    if created:
        channel.save()
        print('Channel {} from {} with id {} added to DB.'.format(channel.name,
                                                                  channel.country_code,
                                                                  channel.id
                                                                  ))
    return channel


def create_showtime(row, program):
    showtime, created = tv_models.Showtime.objects.get_or_create(program=program,
                                                              start_time=row.start_time,
                                                              end_time=row.end_time
                                                              )
    if created:
        showtime.save()
        print('Showtime starting on {} added to '.format(row.start_time)+program.local_title)
    return showtime


# Create Showtimes from DataFrame
def add_showtimes_from_dataframe(df, filename):
    for index, row in df.iterrows():
        channel = create_channel(row)
        program = create_program(row, channel)
        create_showtime(row, program)

    print("Finished adding programs from {}".format(filename))


def main():
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    filename = None
    while filename is None:
        filename = input('Please enter the full path to the Electronic Program Guide: ')
        if os.path.isfile(filename):
            os.chdir(filename[0:-len(filename.split('/')[-1])])
            print ("Uploading programs to database...")
            df = model_for_db(filename.split('/')[-1])
            add_showtimes_from_dataframe(df, filename)
            print ("Finished uploading programs from {} to database.".format(filename))
            return
        else:
            filename = None
            print ("File does not exist. Check the path is correct and try again.")


In [2]:
! ls .
'../../data/epg.csv'

Programs API.ipynb      upload_from_csv.py      vericast_reports.py
__init__.py             upload_from_csv.pyc
__init__.pyc            uploading_epg_csv.ipynb


'../../data/epg.csv'

In [3]:
df = model_for_db('../../data/epg.csv')

In [4]:
add_showtimes_from_dataframe(df, 'epg.csv')

Finished adding programs from epg.csv


In [5]:
df

,channel_id,channel_name,channel_country,start_time,program_id,program_original_title,program_local_title,program_year,end_time
0,384303590,HD1 (France),FRA,2018-01-01 00:00:00,461044,À communiquer,À communiquer,n/a,2018-01-01 23:20:00
1,69036190,La Sexta,ESP,2018-01-01 00:00:00,1887442,Será anunciado,Será anunciado,n/a,2018-01-01 23:45:00
2,69036687,RTE 1,IRL,2018-01-01 00:00:00,3452333967,RTE 1 Programming,RTE 1 Programming,n/a,2018-01-01 23:40:00
3,69036687,RTE 1,IRL,2018-01-01 00:30:00,446381,L.A. Confidential,L.A. Confidential,1997.0,2018-01-01 02:50:00
4,69036190,La Sexta,ESP,2018-01-01 00:45:00,18063328,No controles,No controles,2010.0,2018-01-01 02:20:00
5,69036190,La Sexta,ESP,2018-01-01 02:20:00,5072179538,Canal Ruleta,Canal Ruleta,n/a,2018-01-01 02:50:00
6,69036190,La Sexta,ESP,2018-01-01 02:50:00,11742116,Minutos musicales,Minutos musicales,n/a,2018-01-01 05:00:00
7,69036687,RTE 1,IRL,2018-01-01 02:50:00,1064550,Papillon,Papillon,1973.0,2018-01-01 05:20:00
8,69036190,La Sexta,ESP,2018-01-01 05:00:00,11742116,Minutos musicales,Minutos musicales,n/a,2018-01-01 05:15:00
9,69036190,La Sexta,ESP,2018-01-01 05:15:00,19219963,n/a,Bestial,n/a,2018-01-01 06:07:00
